In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from pprint import pprint 

In [3]:
cities = []
top_100_cities_df = pd.read_csv("csvs/top_100_cities.csv")
for index, row in top_100_cities_df.iterrows():
    cities.append(f"{row['city']}, {row['state_id']}")

In [4]:
search_url = "https://api.teleport.org/api/cities/?search="
#urban_area_url = []

# Criterias that we care about
city_score_dict = {
    'City':[],
    'Housing':[],
    "Cost of Living":[],
    "Startups":[],
    "Travel Connectivity":[],
    "Commute":[],
    "Safety":[],
    "Healthcare":[],
    "Education":[],
    "Environmental Quality":[],
    "Taxation":[],
    "Internet Access":[],
}


city_score_list = []
for city in cities:
    response = requests.get(search_url+city).json()
    city_url = response['_embedded']['city:search-results'][0]['_links']['city:item']['href']
    response = requests.get(city_url).json()
    try:
        urban_area_url=response['_links']['city:urban_area']['href'] +"scores"
        response = requests.get(urban_area_url).json()
        city_score_list.append(response['categories'])
        city_score_dict['City'].append(city)
        for category in response['categories']:
            if category['name'] in city_score_dict.keys():
                city_score_dict[category['name']].append(category['score_out_of_10'])

    except:
        print(city,"doesn't belong to an urban area")



city_df = pd.DataFrame(city_score_dict)

city_df.head()

San Juan, PR doesn't belong to an urban area
Virginia Beach, VA doesn't belong to an urban area
Bridgeport, CT doesn't belong to an urban area
Hartford, CT doesn't belong to an urban area
Tucson, AZ doesn't belong to an urban area
McAllen, TX doesn't belong to an urban area
El Paso, TX doesn't belong to an urban area
Dayton, OH doesn't belong to an urban area
Fresno, CA doesn't belong to an urban area
Tulsa, OK doesn't belong to an urban area
Allentown, PA doesn't belong to an urban area
Cape Coral, FL doesn't belong to an urban area
Springfield, MA doesn't belong to an urban area
Grand Rapids, MI doesn't belong to an urban area
Ogden, UT doesn't belong to an urban area
Columbia, SC doesn't belong to an urban area
Baton Rouge, LA doesn't belong to an urban area
Albany, NY doesn't belong to an urban area
Akron, OH doesn't belong to an urban area
New Haven, CT doesn't belong to an urban area
Provo, UT doesn't belong to an urban area
Bakersfield, CA doesn't belong to an urban area
Toledo,

,City,Housing,Cost of Living,Startups,Travel Connectivity,Commute,Safety,Healthcare,Education,Environmental Quality,Taxation,Internet Access
0,"New York, NY",1.0000,2.342,10.0000,6.6750,5.51925,7.022,8.501667,8.0935,5.23375,3.9205,7.0985
1,"Los Angeles, CA",1.5275,4.556,10.0000,3.9585,3.67700,5.705,8.439667,8.6245,4.73150,4.7675,5.4965
2,"Chicago, IL",3.7020,4.766,9.5130,5.5075,4.57975,3.832,8.555000,7.9990,6.80450,4.0620,6.5995
3,"Miami, FL",3.0310,4.497,8.0225,4.1690,4.05275,4.321,8.529000,4.3945,6.29200,4.7720,5.8240
4,"Dallas, TX",5.2590,6.091,7.9395,4.7465,4.47050,4.339,8.439333,4.2840,6.69300,4.7720,6.4610


In [14]:
city_df["Overall"] = ""
for index, row in city_df.iterrows():
    row[1:-1].astype(float)
    city_df.loc[index, "Overall"] = np.mean(row[1:-1])
city_df.to_csv("csvs/city_scores.csv", index=False)

In [15]:
sorted_overall_df = city_df.sort_values('Overall', ascending=False)
sorted_overall_df

,City,Housing,Cost of Living,Startups,Travel Connectivity,Commute,Safety,Healthcare,Education,Environmental Quality,Taxation,Internet Access,Overall
44,"Raleigh, NC",6.1285,5.610,5.4970,1.7570,4.78425,6.5225,8.966667,7.1165,8.58375,4.0620,7.9515,6.089061
2,"Chicago, IL",3.7020,4.766,9.5130,5.5075,4.57975,3.8320,8.555000,7.9990,6.80450,4.0620,6.5995,5.99275
65,"Worcester, MA",1.0000,4.259,9.5650,3.2150,4.39625,7.7420,8.956000,8.6245,8.17050,4.0620,5.7005,5.971886
9,"Boston, MA",1.0000,4.259,9.5650,3.2150,4.39625,7.7420,8.956000,8.6245,8.17050,4.0620,5.7005,5.971886
0,"New York, NY",1.0000,2.342,10.0000,6.6750,5.51925,7.0220,8.501667,8.0935,5.23375,3.9205,7.0985,5.946015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,"Providence, RI",3.1530,0.000,3.7640,3.0185,2.42200,6.8460,8.885333,5.3015,5.90050,4.2040,8.9345,4.766303
52,"Honolulu, HI",2.3415,2.867,4.1335,1.9830,4.16175,6.8565,8.474000,4.2805,6.09400,4.2040,6.1580,4.686705
20,"Baltimore, MD",4.8300,4.982,0.0000,3.6665,4.93475,2.3730,8.944667,5.8805,5.78450,4.0620,4.9880,4.585992
45,"Memphis, TN",7.1145,5.829,4.4495,1.6615,1.91750,2.8585,7.523667,3.6245,4.54525,4.7720,6.0700,4.57872


In [16]:
# metrics = ["Environmental Quality", "Cost of Living", "Education", "Commute", "Safety"]

In [17]:
sorted_env_df = city_df.sort_values('Environmental Quality', ascending=False)[['City', 'Environmental Quality']]
sorted_env_df.head()

,City,Environmental Quality
69,"Madison, WI",9.52400
64,"Des Moines, IA",8.68225
44,"Raleigh, NC",8.58375
60,"Colorado Springs, CO",8.52650
24,"Portland, OR",8.45950


In [18]:
sorted_CoL_df = city_df.sort_values('Cost of Living', ascending=False)[['City', 'Cost of Living']]
sorted_CoL_df.head()

,City,Cost of Living
25,"San Antonio, TX",6.683
61,"Knoxville, TN",6.447
49,"Oklahoma City, OK",6.394
47,"Louisville, KY",6.358
64,"Des Moines, IA",6.327


In [19]:
sorted_edu_df = city_df.sort_values('Education', ascending=False)[['City', 'Education']]
sorted_edu_df.head()

,City,Education
26,"Sacramento, CA",8.6245
62,"Mission Viejo, CA",8.6245
68,"Long Beach, CA",8.6245
23,"Riverside, CA",8.6245
65,"Worcester, MA",8.6245


In [20]:
sorted_edu_df = city_df.sort_values('Safety', ascending=False)[['City', 'Safety']]
sorted_edu_df.head()

,City,Safety
9,"Boston, MA",7.7420
65,"Worcester, MA",7.7420
69,"Madison, WI",7.3620
50,"Buffalo, NY",7.1885
67,"Staten Island, NY",7.0220


In [21]:
sorted_edu_df = city_df.sort_values('Commute', ascending=False)[['City', 'Commute']]
sorted_edu_df.head()

,City,Commute
46,"Richmond, VA",5.64850
0,"New York, NY",5.51925
17,"Brooklyn, NY",5.51925
67,"Staten Island, NY",5.51925
38,"Bronx, NY",5.51925
